In [ ]:
import numpy as np
from numpy import array
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2
from sympy.matrices import Matrix
import math

In [ ]:
def euler_to_quat(roll, pitch, yaw):
    cy = np.cos(yaw * 0.5)
    sy = np.sin(yaw * 0.5)
    cr = np.cos(roll * 0.5)
    sr = np.sin(roll * 0.5)
    cp = np.cos(pitch * 0.5)
    sp = np.sin(pitch * 0.5)

    qw = cy * cr * cp + sy * sr * sp
    qx = cy * sr * cp - sy * cr * sp
    qy = cy * cr * sp + sy * sr * cp
    qz = sy * cr * cp - cy * sr * sp
    return q

def quat_to_euler(x, y, z, w):
    ysqr = y * y
     
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + ysqr)
    X =  math.atan2(t0, t1)
     
    t2 = +2.0 * (w * y - z * x)
    t2 = -1.0 if t2 < -1.0 else t2
    t2 =  1.0 if t2 >  1.0 else t2
    Y = math.asin(t2)
     
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (ysqr + z * z)
    Z = math.atan2(t3, t4)
     
    return X, Y, Z

def quat_to_rot(qx, qy, qz, qw):
    return np.array([[1 - 2*qy**2 - 2*qz**2 ,2*qx*qy - 2*qz*qw ,2*qx*qz + 2*qy*qw],
            [2*qx*qy + 2*qz*qw ,1 - 2*qx**2 - 2*qz**2 ,2*qy*qz - 2*qx*qw],
            [2*qx*qz - 2*qy*qw ,2*qy*qz + 2*qx*qw ,1 - 2*qx**2 - 2*qy**2]])


#### ---------------------------------------------------------------------------------------------------------------------------------------------------------
#### *Solve Forward kinematics*
#### ---------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
### symbols of joint variables
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') #theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')


In [ ]:
# DH Parameters
s = {alpha0:     0, a0:      0, d1:  0.75,
     alpha1: -pi/2, a1:   0.35, d2:     0,   q2: q2 - pi/2,
     alpha2:     0, a2:   1.25, d3:     0,
     alpha3: -pi/2, a3: -0.054, d4:  1.50,
     alpha4:  pi/2, a4:      0, d5:     0,
     alpha5: -pi/2, a5:      0, d6:     0,
     alpha6:     0, a6:      0, d7: 0.303,   q7: 0}

In [ ]:
#### Homogeneous Transforms
# base_link to link1
T0_1 = Matrix([[             cos(q1),            -sin(q1),            0,              a0],
               [ sin(q1)*cos(alpha0), cos(q1)*cos(alpha0), -sin(alpha0), -sin(alpha0)*d1],
               [ sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),  cos(alpha0),  cos(alpha0)*d1],
               [                   0,                   0,            0,               1]])
T0_1 = T0_1.subs(s)

#link1 to link2
T1_2 = Matrix([[             cos(q2),            -sin(q2),            0,              a1],
               [ sin(q2)*cos(alpha1), cos(q2)*cos(alpha1), -sin(alpha1), -sin(alpha1)*d2],
               [ sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),  cos(alpha1),  cos(alpha1)*d2],
               [                   0,                   0,            0,               1]])
T1_2 = T1_2.subs(s)

T2_3 = Matrix([[             cos(q3),            -sin(q3),            0,              a2],
               [ sin(q3)*cos(alpha2), cos(q3)*cos(alpha2), -sin(alpha2), -sin(alpha2)*d3],
               [ sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),  cos(alpha2),  cos(alpha2)*d3],
               [                   0,                   0,            0,               1]])
T2_3 = T2_3.subs(s)

T3_4 = Matrix([[             cos(q4),            -sin(q4),            0,              a3],
               [ sin(q4)*cos(alpha3), cos(q4)*cos(alpha3), -sin(alpha3), -sin(alpha3)*d4],
               [ sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),  cos(alpha3),  cos(alpha3)*d4],
               [                   0,                   0,            0,               1]])
T3_4 = T3_4.subs(s)

T4_5 = Matrix([[             cos(q5),            -sin(q5),            0,              a4],
               [ sin(q5)*cos(alpha4), cos(q5)*cos(alpha4), -sin(alpha4), -sin(alpha4)*d5],
               [ sin(q5)*sin(alpha4), cos(q5)*sin(alpha4),  cos(alpha4),  cos(alpha4)*d5],
               [                   0,                   0,            0,               1]])
T4_5 = T4_5.subs(s)

T5_6 = Matrix([[             cos(q6),            -sin(q6),            0,              a5],
               [ sin(q6)*cos(alpha5), cos(q6)*cos(alpha5), -sin(alpha5), -sin(alpha5)*d6],
               [ sin(q6)*sin(alpha5), cos(q6)*sin(alpha5),  cos(alpha5),  cos(alpha5)*d6],
               [                   0,                   0,            0,               1]])
T5_6 = T5_6.subs(s)

T6_G = Matrix([[             cos(q7),            -sin(q7),            0,              a6],
               [ sin(q7)*cos(alpha6), cos(q7)*cos(alpha6), -sin(alpha6), -sin(alpha6)*d7],
               [ sin(q7)*sin(alpha6), cos(q7)*sin(alpha6),  cos(alpha6),  cos(alpha6)*d7],
               [                   0,                   0,            0,               1]])
T6_G = T6_G.subs(s)

In [ ]:
# Corerection rotation matrix for the difference between DH and urdf reference frames for the gripper link
R_z = Matrix([[     cos(np.pi), -sin(np.pi),             0,   0],
              [     sin(np.pi),  cos(np.pi),             0,   0],
              [              0,           0,             1,   0],
              [              0,           0,             0,   1]])
 
R_y = Matrix([[  cos(-np.pi/2),           0, sin(-np.pi/2),   0],
              [              0,           1,             0,   0],
              [ -sin(-np.pi/2),           0, cos(-np.pi/2),   0],
              [              0,           0,       0,         1]])
R_corr = R_z * R_y

In [ ]:
# Homogeneous transformation from base link to gripper frame 
T0_G = simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_G * R_corr)

In [ ]:
# first test case
q1_v = -0.65
q2_v = 0.45
q3_v = -0.36
q4_v = 0.95
q5_v = 0.79
q6_v = 0.49

# substitute symbolic variables by test case values
T0_G_val = T0_G.evalf(subs={q1: q1_v, q2: q2_v, q3: q3_v, q4: q4_v, q5: q5_v, q6: q6_v})
T0_G_val = np.array(T0_G_val.tolist(), np.float)
print("Transformation Matrix from base to end effector= ")
print(T0_G_val)
print()
print("Translation = ", T0_G_val[:3,3])
print("roll = ", np.arctan2(T0_G_val[2,1], T0_G_val[2,2])) #roll
print("pitch = ", np.arctan2(-T0_G_val[2,0], np.sqrt(T0_G_val[0,0]**2 + T0_G_val[1,0]**2))) #pitch
print("yaw = ",np.arctan2(T0_G_val[1,0], T0_G_val[0,0])) #yaw


#### ---------------------------------------------------------------------------------------------------------------------------------------------------------
#### *Solve Inverse kinematics*
#### ---------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
test_cases = {1:[[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]],
              2:[[[-0.56754,0.93663,3.0038],
                  [0.62073, 0.48318,0.38759,0.480629]],
                  [-0.638,0.64198,2.9988],
                  [-0.79,-0.11,-2.33,1.94,1.14,-3.68]],
              3:[[[-1.3863,0.02074,0.90986],
                  [0.01735,-0.2179,0.9025,0.371016]],
                  [-1.1669,-0.17989,0.85137],
                  [-2.99,-0.12,0.94,4.06,1.29,-4.12]],
              4:[[[-0.569, 0.572, 3.384],
                  [0.350, 0.842, 0.157, -0.378]],
                  [-0.42701, 0.42965, 3.1575],
                  [-0.788488, 0, -2.296801, 0, 0, 0]],
              5:[[[2.153, 0.000, 1.947],
                  [0.000, -0.000, 0.000, 1.000]],
                  [1.8499, 0, 1.9464],
                  [0,0,0,0,0,0]]}

ind = 1
joints = test_cases[ind][2]
position = test_cases[ind][0][0]
orientation   =  test_cases[ind][0][1]

In [ ]:
R = quat_to_rot(*orientation)
wc = position - (0.193 + 0.11) * R[:3,0]
print(wc)

In [ ]:
theta1 = np.arctan2(wc[1], wc[0])
print(theta1)

In [ ]:
wc_x = np.sqrt(wc[0]**2+wc[1]**2)
wc_y = wc[2]

l2_x = 0.35
l2_y = 0.33 + 0.42

l2_l3 = 1.25
l2_wc = np.sqrt((wc_x - l2_x)**2 + (wc_y - l2_y)**2)
l3_wc = np.sqrt((0.96+0.54)**2+0.054**2)

In [ ]:
theta2 = np.pi/2 - np.arctan2(wc_y - l2_y, wc_x - l2_x) - np.arccos((l2_wc**2+l2_l3**2-l3_wc**2)/(2*l2_wc*l2_l3))
print(theta2)

In [ ]:
theta3 = np.pi/2 - np.arctan2(0.054, 0.96+0.54) \
    - np.arccos((l3_wc**2+l2_l3**2-l2_wc**2)/(2*l3_wc*l2_l3))
print(theta3)

In [ ]:
R_z = Matrix([[   np.cos(theta1),-np.sin(theta1),             0],
              [   np.sin(theta1), np.cos(theta1),             0],
              [             0,           0,             1]])

R_y =  Matrix([[  np.cos(theta2+theta3),           0, np.sin(theta2+theta3)],
                 [              0,           1,             0],
                 [ -np.sin(theta2+theta3),           0, np.cos(theta2+theta3)]])

R_b3 = np.array((R_z * R_y).tolist(), np.float)
print(R_b3)

In [ ]:
R_x1 = Matrix([[   1,        0,        0],
               [   0,  cos(q4), -sin(q4)],
               [   0,  sin(q4),  cos(q4)]])

R_y =  Matrix([[  cos(q5),           0, sin(q5)],
               [        0,           1,       0],
               [ -sin(q5),           0, cos(q5)]])

R_x2 = Matrix([[   1,        0,        0],
               [   0,  cos(q6), -sin(q6)],
               [   0,  sin(q6),  cos(q6)]])


R = simplify(R_x1 * R_y * R_x2)
R

In [ ]:
R_n = np.array((Matrix(R_b3).T * Matrix(quat_to_rot(*orientation))).tolist(), float)
R_n

In [ ]:
theta5 = np.arctan2(np.sqrt(R_n[1, 0] ** 2 + R_n[2, 0] ** 2), R_n[0, 0])
print(theta5)

In [ ]:
theta6 = np.arctan2(R_n[0,1], R_n[0,2])
print(theta6)

In [ ]:
theta4 = np.arctan2(R_n[1, 0], -R_n[2, 0])
print(theta4)

In [ ]:
[theta1, theta2, theta3, theta4, theta5, theta6]

In [ ]:
joints